In [1]:
import pandas as pd
import numpy as np

**read_data_small** is the function to read in the small dataset about 30 MB

In [2]:
def read_data_small():
    X_train = pd.read_csv("data_small/X_train_small.csv")
    X_test = pd.read_csv("data_small/X_test_small.csv")
    y_train = np.asarray(pd.read_csv("data_small/y_train_small.csv", header=None)[0])
    # y_train_masked = np.asarray(pd.read_csv("data_small/y_masked_train_small.csv", header=None)[0])
    return X_train, X_test, y_train

**read_data_big** is the function to read in the big dataset about 100 MB

In [3]:
def read_data_big():
    X_train = pd.read_csv("data_big/X_train_big.csv")
    X_test = pd.read_csv("data_big/X_test_big.csv")
    y_train = np.asarray(pd.read_csv("data_big/y_train_big.csv", header=None)[0])
    # y_train_masked = np.asarray(pd.read_csv("data_big/y_masked_train_big.csv", header=None)[0])
    return X_train, X_test, y_train

**read_data** is the function to read in the whole dataset about 1.5 G

In [4]:
def read_data():
    X_train = pd.read_csv("data/X_train.csv")
    X_test = pd.read_csv("data/X_test.csv")
    y_train = np.asarray(pd.read_csv("data/y_train.csv", header=None)[0])
    # y_train_masked = np.asarray(pd.read_csv("data/y_masked_train.csv", header=None)[0])
    return X_train, X_test, y_train

# Insert Your Code Here

**detect_spoofying** is the function for training the classifier and classify the results. 

Here we provide an simple example.

In [5]:
### import libraries here ###
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale

### code classifier here ###
def format_data(df):
    # append numberical columns
    rst = df.loc[:,["price","volume","bestBid","bestAsk",'bestBidVolume',
                    'bestAskVolume','lv2Bid', 'lv2BidVolume','lv2Ask', 
                    'lv2AskVolume', 'lv3Bid', 'lv3BidVolume', 'lv3Ask',
                    'lv3AskVolume']]
    # encode the binaries
    rst["isBid"] = df.isBid*1
    rst["isBuyer"] = df.isBuyer*1
    rst["isAggressor"] = df.isAggressor*1
    rst["type"] = (df.type == "ORDER")*1
    rst["source"] = (df.source=="USER")*1
    # parse the order id data
    rst["orderId"] = df.orderId.str.split('-').str[-1]
    rst["tradeId"] = df.tradeId.str.split('-').str[-1]
    rst["bidOrderId"] = df.bidOrderId.str.split('-').str[-1]
    rst["askOrderId"] = df.askOrderId.str.split('-').str[-1]
    # encode the multiple lable data
    tmp_operation = pd.DataFrame(pd.get_dummies(df.operation), columns=df.operation.unique()[:-1])
    rst = pd.concat([rst, tmp_operation], axis=1)
    tmp_endUserRef = pd.DataFrame(pd.get_dummies(df.endUserRef), columns=df.endUserRef.unique()[:-1])
    rst = pd.concat([rst, tmp_endUserRef], axis=1)
    return rst

def detect_spoofying(X_train, X_test, y_train):
    # clean up the data
    X_clean = format_data(pd.concat([X_train, X_test]))
    X_clean = X_clean.fillna(-1)
    X_train_clean = X_clean.iloc[:X_train.shape[0],:]
    X_test_clean = X_clean.iloc[X_train.shape[0]:,:]
    X_train_clean_scaled = scale(X_train_clean)
    X_test_clean_scaled = scale(X_test_clean)

    # fit classifier
    clf = LogisticRegression(random_state=0, class_weight='balanced').fit(X_train_clean_scaled, y_train)
    y_train_prob_pred = clf.predict_proba(X_train_clean_scaled)
    y_test_prob_pred = clf.predict_proba(X_test_clean_scaled)
    return y_train_prob_pred, y_test_prob_pred

**score** is the function that we use to compare the results. An example is provided with scoring the predictions for the training dataset. True labels for the testing data set will be supplied to score the predictions for testing dataset.

Score is based on cohen's kappa measurement. https://en.wikipedia.org/wiki/Cohen%27s_kappa

In [6]:
from sklearn.metrics import cohen_kappa_score

def score(y_pred, y_true):
    """
    y_pred: a numpy 4d array of probabilities of point assigned to each label
    y_true: a numpy array of true labels
    """
    y_pred_label = np.argmax(y_pred, axis=1)
    return cohen_kappa_score(y_pred_label, y_true)

**wrapper** is the main function to read in unzipped data and output a score for evaluation. In addition, the function returns the y probability matrix (both train and test) for grading. More details about submitting format are outlined below.

In [7]:
def wrapper():
    # read in data
    X_train, X_test, y_train = read_data_small()
    # or if you have the computational power to work with the big data set, 
    # you can comment out the read_data_samll line and uncomment the following read_data_big
    # X_train, X_test, y_train = read_data_big()
    
    # process the data, train classifier and output probability matrix
    y_train_prob_pred, y_test_prob_pred = detect_spoofying(X_train, X_test, y_train)
    
    # score the predictions
    score_train = score(y_train_prob_pred, y_train)
    # score_test = score(y_test_prob_pred, y_test)
    
    # return the scores
    return score_train, y_train_prob_pred, y_test_prob_pred

Call function wrapper:

In [8]:
score_train, y_train_prob_pred, y_test_prob_pred = wrapper()

C:\Users\aliencommander\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Score for training data set is:

In [9]:
score_train

0.4927822399601266

### Submission Format

The classifier function wrote should return a 4d nparray with 4 columns. The columns are corresponding to the class labels: 0, 1, 2, 3. Please see examples below.

In [10]:
y_train_prob_pred

array([[1.00000000e+000, 2.38578272e-043, 1.68089121e-111,
        1.67989132e-050],
       [1.00000000e+000, 9.45486066e-045, 4.10284863e-113,
        2.69294241e-052],
       [1.00000000e+000, 7.90069998e-045, 4.57559128e-113,
        3.59863525e-052],
       ...,
       [9.75809246e-001, 3.96899900e-003, 5.02811256e-003,
        1.51936424e-002],
       [9.99295974e-001, 1.14132011e-004, 1.18484560e-004,
        4.71409182e-004],
       [9.99547411e-001, 2.13782709e-004, 9.31318636e-005,
        1.45674867e-004]])

In [11]:
y_test_prob_pred

array([[9.99595739e-01, 1.26064722e-04, 2.65508889e-05, 2.51645227e-04],
       [9.99643860e-01, 1.10954373e-04, 5.67291413e-05, 1.88456225e-04],
       [8.56739642e-01, 1.35267719e-01, 5.35675771e-03, 2.63588132e-03],
       ...,
       [9.80024532e-01, 5.01997107e-04, 1.90111960e-02, 4.62274753e-04],
       [9.73707380e-01, 3.83413743e-04, 2.56717712e-02, 2.37435408e-04],
       [9.71176872e-01, 9.73269116e-04, 2.78063139e-02, 4.35451462e-05]])

### Write test results to csv files

Please rename your file to indicate which data set you are working with. 

- If you are using the small dataset: *y_train_prob_pred_small.csv* and *y_test_prob_pred_small.csv*
- If you are using the small dataset: *y_train_prob_pred_big.csv* and *y_test_prob_pred_big.csv*
- If you are using the original dataset: *y_train_prob_pred.csv* and *y_test_prob_pred.csv*

In [12]:
pd.DataFrame(y_train_prob_pred).to_csv("teamwh_y_train_prob_pred_small.csv")
pd.DataFrame(y_test_prob_pred).to_csv("teamwh_y_test_prob_pred_small.csv.csv")

In [ ]:
from zipfile import ZipFile
#create a ZipFile object
zipObj = ZipFile('teamwh_small.zip', 'w')
 
# Add multiple files to the zip
zipObj.write('teamwh.ipynb')
zipObj.write('teamwh_y_train_prob_pred_small.csv')
zipObj.write('teamwh_y_test_prob_pred_small.csv')

# close zip file
zipObj.close()

